In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('stopwords')

df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

[nltk_data] Downloading package stopwords to /home/tomas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def ver_info_a_filtrar(df, col):
    cant_publicaciones_columna_no_nulo = df[~df[col].isnull()].shape[0]
    print('La cantidad de publicaciones con ' + col +' no nulo es:', cant_publicaciones_columna_no_nulo)
    print('La cantidad total de publicaciones es:', df.shape[0])

In [3]:
def generar_palabras_no_queridas(arr=None):
    '''Retorna un set con las palabras a excluir, arr, es un campo opcional para excluir tambien 
    las palabras en ese arreglo'''
    stop_words = set(stopwords.words('spanish'))
    puntuacion = [',', '.', '!', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '-', '_', '\'', '\"', 
                  '(', ')', '[', ']', '{', '}', 'i', 'p', 'br', '<', '>', 'col.', '\\', '/', '$', 'm2', 
                  '..', '...', 'ii', '#', ';', ':', 'col']
    for x in puntuacion:
        stop_words.add(x)
    if arr:
        for x in arr:
            stop_words.add(x)
    return stop_words

In [4]:
def generar_palabras_mas_frecuentes(df, col, n):
    '''Esta funcion recibe un df, y genera la n palabras mas usadas la columna pasada por parametro, 
    retorna un arreglo ordenado de mas usadas a menos usadas'''
    reviews = df[col].str.cat(sep=' ')

    # Splitea los textos en palabras
    tokens = word_tokenize(reviews)

    # Fuera palabras no necesarias
    stop_words = generar_palabras_no_queridas()
    tokens = [w for w in tokens if not w in stop_words]

    vocabulary = set(tokens)
    
    # Calcula y ordena por frecuencia
    frequency_dist = nltk.FreqDist(tokens)
    mas_usadas = sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)[0:n]
    return mas_usadas

In [5]:
def feature_cantidad_mas_frecuentes(df, col, n):
    '''Retorna el df pasado por parametro con la columna nueva "cant_palabras_mas_frecuentes", donde dice
    el numero de palabras mas frecuentes presentes en la columna pasada por parametro. n es la cantidad 
    palabras mas frecuentes a usar. Por ejemplo: `feature_cantidad_mas_frecuentes(df, 'titulo', 50)` genera 
    columna nueva `cant_palabras_mas_frecuentes_titulo`, donde se indica cuantas palabras frecuentes aparecen
    en el titulo, siendo las palabras mas frecuentes, la n palabras mas usadas entre todas las columnas 
    pasadas por parametro'''
    
    ver_info_a_filtrar(df, col)
    
    arr_mas_frecuentes = generar_palabras_mas_frecuentes(df, col, n)
    set_mas_frecuentes = set(arr_mas_frecuentes)
    
    def contador_mas_frecuentes(texto):
        contador = 0
        palabras_del_df = word_tokenize(texto)
        for palabra in palabras_del_df:
            if palabra in set_mas_frecuentes:
                contador = contador + 1
        return contador
    
    df['cant_palabras_mas_frecuentes_' + col] = df[~df[col].isnull()][col].apply(contador_mas_frecuentes)
    df['cant_palabras_mas_frecuentes_' + col] = df['cant_palabras_mas_frecuentes_' + col].fillna(0)
    
    return df

In [ ]:
def ejemplo():
    # Ejemplo de uso de las palabras mas frecuentes
    df_palabras = feature_cantidad_mas_frecuentes(df_train, 'titulo', 200)
    df_palabras[['id', 'titulo', 'cant_palabras_mas_frecuentes_titulo']].head()

In [6]:
def generar_palabras_menos_frecuentes(df, col, n):
    '''Esta funcion recibe un df, y genera la n palabras menos usadas la columna pasada por parametro, 
    retorna un arreglo ordenado de menos usadas a menos usadas'''
    reviews = df[col].str.cat(sep=' ')

    # Splitea los textos en palabras
    tokens = word_tokenize(reviews)

    # Fuera palabras no necesarias
    stop_words = generar_palabras_no_queridas()
    tokens = [w for w in tokens if not w in stop_words]

    vocabulary = set(tokens)
    
    # Calcula y ordena por frecuencia
    frequency_dist = nltk.FreqDist(tokens)
    menos_usadas = sorted(frequency_dist, key=frequency_dist.__getitem__, reverse=False)[0:n]
    return menos_usadas

In [7]:
def feature_cantidad_menos_frecuentes(df, col, n):
    '''Retorna el df pasado por parametro con la columna nueva "cant_palabras_menos_frecuentes", donde dice
    el numero de palabras menos frecuentes presentes en la columna pasada por parametro. n es la cantidad 
    palabras menos frecuentes a usar. Por ejemplo: `cant_palabras_menos_frecuentes(df, 'titulo', 50)` genera 
    columna nueva `cant_palabras_menos_frecuentes_titulo`, donde se indica cuantas palabras menos frecuentes aparecen
    en el titulo, siendo las palabras menos frecuentes, la n palabras menos usadas entre todas las columnas 
    pasadas por parametro'''
    
    ver_info_a_filtrar(df, col)
    
    arr_menos_frecuentes = generar_palabras_menos_frecuentes(df, col, n)
    set_menos_frecuentes = set(arr_menos_frecuentes)
    
    def contador_menos_frecuentes(texto):
        contador = 0
        palabras_del_df = word_tokenize(texto)
        for palabra in palabras_del_df:
            if palabra in set_menos_frecuentes:
                contador = contador + 1
        return contador
    
    df['cant_palabras_menos_frecuentes_' + col] = df[~df[col].isnull()][col].apply(contador_menos_frecuentes)
    df['cant_palabras_menos_frecuentes_' + col] = df['cant_palabras_menos_frecuentes_' + col].fillna(0)
    
    return df

In [ ]:
def ejemplo():
    # Ejemplo de uso de las palabras menos frecuentes
    df_palabras = feature_cantidad_menos_frecuentes(df_train, 'titulo', 200)
    df_palabras[['id', 'titulo', 'cant_palabras_menos_frecuentes_titulo']].head()

In [8]:
def palabras_mas_usadas_en_mas_caros_baratos(df, col, n_palabras, n_caros_baratos, df_test=None, mirar_mas_caros=True):
    """Busca los n_caros mas caros del df, y verifica las n_palabras mas usadas."""
    """Para cada texto de la columna col, cuenta cuantas de estas palabras contiene"""
    """Si se pasa un df_test, hace la cuenta teniendo en cuenta las palabras obtenidas en df"""
    """Si mirar_mas_caros es True, va a buscar las palabras mas frecuentos de los mas caros, si fuera False,
    buscaria las palabras mas frecuentes de los mas baratos"""
    
    info = 'CAROS' if mirar_mas_caros else 'BARATOS'
    
    ver_info_a_filtrar(df, col)
    df_busqueda = df.nlargest(n_caros_baratos, 'precio') if mirar_mas_caros else df.nsmallest(n_caros_baratos, 'precio')
    arr_mas_frecuentes = generar_palabras_mas_frecuentes(df_busqueda, col, n_palabras)
    set_mas_frecuentes = set(arr_mas_frecuentes)

    def contador_mas_frecuentes(texto):
        contador = 0
        palabras_del_df = word_tokenize(texto)
        for palabra in palabras_del_df:
            if palabra in set_mas_frecuentes:
                contador = contador + 1
        return contador
    
    df['palabras_mas_frecuentes_' + info + '_' + col] = df[~df[col].isnull()][col].apply(contador_mas_frecuentes)
    df['palabras_mas_frecuentes_' + info + '_' + col] = df['palabras_mas_frecuentes_' + info + '_' + col].fillna(0)
    
    if df_test is None:
        return df, df_test
    
    df_test['palabras_mas_frecuentes_' + info + '_' + col] = df_test[~df_test[col].isnull()][col].apply(contador_mas_frecuentes)
    df_test['palabras_mas_frecuentes_' + info + '_' + col] = df_test['palabras_mas_frecuentes_' + info + '_' + col].fillna(0)
    
    return df, df_test

In [ ]:
def ejemplo():
    df = df_train
    df_aut = df_test
    palabras_mas_usadas_en_mas_caros_baratos(df, 'titulo', 80, 200, df_aut, False)

In [9]:
#Es mas lenteja

def mas_frecuentes_caros_baratos_ohe(df, col, n_palabras, n_caros_baratos, df_test=None, mirar_mas_caros=True):
    """Busca los n_caros mas caros del df, y verifica las n_palabras mas usadas."""
    """Hace ohe de las palabras mas usadas obtenidas, sobre la columna col"""
    """Si se pasa un df_test, hace la cuenta teniendo en cuenta las palabras obtenidas en df"""
    """Si mirar_mas_caros es True, va a buscar las palabras mas frecuentos de los mas caros, si fuera False,
    buscaria las palabras mas frecuentes de los mas baratos"""
    
    def fill_nans(df, columns, value):
        for column in columns:
            df[column] = df[column].fillna(value)
        return df
    
    ver_info_a_filtrar(df, col)
    df_busqueda = df.nlargest(n_caros_baratos, 'precio') if mirar_mas_caros else df.nsmallest(n_caros_baratos, 'precio')
    arr_mas_frecuentes = generar_palabras_mas_frecuentes(df_busqueda, col, n_palabras)
    df[col] = df[col].fillna('vacio')
    cv = CountVectorizer(vocabulary=arr_mas_frecuentes)    
    r = pd.SparseDataFrame(cv.fit_transform(df[col]), df['id'], cv.get_feature_names(), default_fill_value=0)
    r = r.reset_index()
    df_merge = df.merge(r, on='id')
    df_merge = fill_nans(df_merge, cv.get_feature_names(), 0)
    
    if df_test is None:
        return df_merge, df_test
    
    df_test[col] = df_test[col].fillna('vacio')
    r_test = pd.SparseDataFrame(cv.fit_transform(df_test[col]), df_test['id'], cv.get_feature_names(), default_fill_value=0)
    r_test = r_test.reset_index()
    df_merge_aux = df_test.merge(r_test, on='id')
    df_merge_aux = fill_nans(df_merge_aux, cv.get_feature_names(), 0)
    
    return df_merge, df_merge_aux

In [ ]:
def ejemplo():
    df = df_train
    df_aut = df_test
    df_merge, df_merge_aux = mas_frecuentes_caros_baratos_ohe(df, 'titulo', 200, 1000, df_aut, True)

La cantidad de publicaciones con titulo no nulo es: 234613
La cantidad total de publicaciones es: 240000


/home/tomas/Escritorio/kmeans/datos-tp2/.venv/lib/python3.6/site-packages/ipykernel_launcher.py:20: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

/home/tomas/Escritorio/kmeans/datos-tp2/.venv/lib/python3.6/site-packages/pandas/core/sparse/frame.py:257: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  sparse_index=BlockIndex(N, blocs, blens),
/home/tomas/Escritorio/kmeans/datos-tp2/.venv/lib/python3.6/site-packages/pandas/core/sparse/frame.py:269: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> 